# AI vs. Human
Assuming the same image segmenter and models, results are predetermined for labeling.
Thus, we can easily compare on an image-to-image basis the assigned Region and its ML-assigned label ("Labels") and compare against the human assigned labels ("Labels_RC" for Rachel, can change based on person)